In [1]:
import requests # for web scraping
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta, datetime
import numpy as np

# sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from sentiment_dictionary import NEW_WORDS

# install yahoo finance
import yfinance as yf

from config import save_path

# import functions
from functions import scrape_finviz, get_stock_prices

In [2]:
# scrape news articles
news = scrape_finviz(['GOOG', 'AAPL'])
print(news.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   company   193 non-null    object
 1   date      193 non-null    object
 2   time      193 non-null    object
 3   headline  193 non-null    object
dtypes: object(4)
memory usage: 6.2+ KB
None


In [3]:
# get stock prices
data = get_stock_prices(news)

In [7]:
data['time'][0]

'04:25PM'

In [8]:
data['date'][0]+data['time'][0]

'Apr-21-2104:25PM'